# Fetch Scopus Data

Resouces:

- [Scopus Search API](https://dev.elsevier.com/documentation/ScopusSearchAPI.wadl)
- [Scopus Retrieval of more than 5,000 artilcles](https://dev.elsevier.com/support.html)(Q: How can I obtain more than 5,000 / 6,000 results through Scopus / ScienceDirect APIs?)
- [Interactive Scopus API](https://dev.elsevier.com/scopus.html)
- [API Settings (rate limits)](https://dev.elsevier.com/api_key_settings.html)
- Remember Logging In to Cisco VPN!!!


In [1]:
from src.data.ScopusArticleFetcher import ScopusArticleFetcher

In [5]:
import json

path = "../notebooks/api_key_scopus.json"
api_key = json.load(open(path))
api_key = api_key["api_key"]
saving_interval = 250
fetcher = ScopusArticleFetcher(api_key)
full_df = fetcher.fetch_results(saving_interval)

Number of results: 42305
Number of iterations: 1693


Fetching results: 100%|██████████| 1693/1693 [39:44<00:00,  1.41s/iteration] 


Saved final results to final_scopus_results.csv


In [9]:
pd.to_datetime(full_df["prism:coverDate"]).dt.year.value_counts().sort_index()

prism:coverDate
1982      79
1983      97
1984     105
1985     135
1986     111
1987     134
1988     182
1989     279
1990     348
1991     436
1992     543
1993     612
1994     683
1995     725
1996     831
1997     950
1998     942
1999    1012
2000    1003
2001    1044
2002    1124
2003    1130
2004    1277
2005    1251
2006    1376
2007    1356
2008    1422
2009    1371
2010    1451
2011    1424
2012    1517
2013    1520
2014    1514
2015    1630
2016    1645
2017    1458
2018    1482
2019    1490
2020    1553
2021    1738
2022    1704
2023    1621
Name: count, dtype: int64